## Install the matrice package or upgrade it if already installed

In [ ]:
#!python -m pip install --upgrade --index-url https://test.pypi.org/simple/ --no-deps matrice
#!python -m pip install --upgrade matrice

In [ ]:
!python -m pip install --upgrade matrice

## Install required dependencies

In [ ]:
!pip install opendatasets

In [ ]:
!pip install pycocotools pyyaml seaborn

## Setting up the credentials
*   ENV
*   MATRICE_ACCESS_KEY_ID
*   MATRICE_SECRET_ACCESS_KEY
*   MATRICE_ACCOUNT_NUMBER

In [ ]:
import yaml
import os

class Vars:
    _data = {}
    _path = None

    @classmethod
    def initialize(cls, path):
        cls._path = path
        cls._data = cls.load()

    @classmethod
    def load(cls):
        if cls._path and os.path.exists(cls._path):
            with open(cls._path, 'r') as f:
                return yaml.safe_load(f)
        return {}

    @classmethod
    def get(cls, key):
        if key not in cls._data:
            raise KeyError(f"Variable {key} not found")
        return cls._data[key]

    @classmethod
    def set(cls, key, value):
        cls._data[key] = value
        with open(cls._path, 'w') as f:
            yaml.safe_dump(cls._data, f, default_flow_style=False)


In [ ]:
Vars.initialize('vars.yaml')

In [ ]:
Vars.set("ENV", "prod")
Vars.set("MATRICE_ACCESS_KEY_ID", "YU9OZ4AE*******GM8R6H")
Vars.set("MATRICE_SECRET_ACCESS_KEY", "8DBFKOIMG*******0V9B5")
Vars.set("MATRICE_ACCOUNT_NUMBER", "549019934232*******03509")

In [ ]:
import os
os.environ['ENV'] = Vars.get('ENV')
access_key = Vars.get('MATRICE_ACCESS_KEY_ID')
secret_key = Vars.get('MATRICE_SECRET_ACCESS_KEY')
account_number = Vars.get('MATRICE_ACCOUNT_NUMBER')

## Import required matrice modules

In [ ]:
from matrice.session import Session
from matrice.projects import Projects
from matrice.dataset import Dataset
from matrice.models import Model
from matrice.deployment import Deployment
from matrice.exported_model import ExportedModel
import sys
import os

# Create Object Detection Project with Matrice SDK

1.   Initialize matrice connection session
2.   List projects in user account
3.   Create the Project
4.   Update the session with the project_id

In [ ]:
session_instance = Session.create_session(account_number=account_number, access_key=access_key, secret_key=secret_key)
print("A Session has been initialized:", session_instance)

In [ ]:
projects_list, msg = session_instance.list_projects(project_type="detection")

# Check if there is a message or error
if msg:
    print(f"Message: {msg}")

# Print the projects in a formatted manner
for project_name, project_instance in projects_list.items():
    print(f"Project Name: {project_name} | Project ID: {project_instance.project_id}")

In [ ]:
project_name = "Vehicles-licence-plate-detection"
for name, project_instance in projects_list.items():
    if name == project_name:
        projects_instance = Projects(session_instance, project_name=project_name)
        session_instance.update(projects_instance.project_id)
        print(f"Project '{project_name}' already exists. Initialized Projects instance.")
        project_exists = True
        break
    else:
        project_exists = False

if project_exists == False:
    print(f"Creating a new project: {project_name}")


In [ ]:
import pprint
name = "Vehicles-licence-plate-detection"
# Function to create a project
def create_project():
    project_name = name
    input_type = "image"
    output_type = "detection"

    enabled_platforms = {
        "matrice": True,
        "android": False,
        "ios": False,
        "tpu": False,
        "intelCPU": False,
        "gcloudGPU": False
    }

    resp, error = session_instance._create_project(project_name, input_type, output_type)
    if error:
        print(f"Error: {error}")
        return None, None
    else:
        print(f"Project created with ID: {resp['_id']}")
        return resp['_id'], resp['name']

# Check if project_id and project_name exist
project_id = None
project_name = None

if not project_exists:
    project_id, project_name = create_project()
else:
    project_id = projects_instance.project_id
    project_name = projects_instance.project_name
    print(f"Project already exists with ID: {project_id} and Name: {project_name}")


print("----------------------------")
print(f"Project ID: {project_id}")
print(f"Project Name: {project_name}")
print("----------------------------")


In [ ]:
Vars.set("project_name",project_name)
Vars.set("project_id", project_id)

In [ ]:
session_instance.update(project_id)
project_instance = Projects(session_instance, project_name=project_name)
print("A Project class has been initialized",project_instance)

# Preparing the Dataset


1.   Dataset Downlaod (either local download or use scripts to download)
2.   Dataset Exploration
3.   Plot Samples from the Dataset
4.   Dataset Format Preparation in MSCOCO
5.   Compress the Dataset to be ready for uploading

In [ ]:
from opendatasets import download
download("https://www.kaggle.com/datasets/amirhoseinahmadnejad/car-license-plate-detection-iran")

In [ ]:
import json
import pandas as pd

# Load COCO JSON file
with open('/content/car-license-plate-detection-iran/train/_annotations.coco.json', 'r') as f:
    coco_data = json.load(f)

# Extract relevant information
annotations = coco_data['annotations']
images = coco_data['images']
categories = coco_data['categories']

# Convert annotations to DataFrame
df_annotations = pd.DataFrame(annotations)

# Convert images to DataFrame
df_images = pd.DataFrame(images)

# Merge annotations and images based on image_id
df = pd.merge(df_annotations, df_images, left_on='image_id', right_on='id')

# Add region_count by counting occurrences of each image in the annotations
df['region_count'] = df.groupby('image_id')['image_id'].transform('count')

# Inspect the DataFrame
df.head()

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns

# Visualize region counts by category (you may need to adjust this if different structure)
category_counts = df['category_id'].value_counts()

# Plot the counts
category_counts.plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right']].set_visible(False)
plt.title("Category Counts")
plt.show()

In [ ]:
# Drop duplicates and keep unique images
df_unique = df.drop_duplicates(subset=['file_name'])

# Plot histogram of region counts
df_unique['region_count'].plot(kind='hist', bins=20, title='Region Count per Image')
plt.gca().spines[['top', 'right']].set_visible(False)
plt.show()

In [ ]:
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import json

def plot_samples_from_coco(df_annotations, df_images, num_samples, image_path, images_per_row=3):
    """
    Plots bounding boxes from COCO annotations on the specified images side by side with reduced spacing.

    Parameters:
    - df_annotations: DataFrame containing the COCO annotations
    - df_images: DataFrame containing the COCO image details (file names, IDs)
    - num_samples: Number of the file names to plot
    - image_path: Path to the directory containing the image files
    - images_per_row: Number of images to display per row
    """
    # Ensure num_samples does not exceed the number of unique filenames in the DataFrame
    unique_file_names = df_images['file_name'].unique()
    num_samples = min(num_samples, len(unique_file_names))

    # Randomly sample file names
    file_names = np.random.choice(unique_file_names, num_samples, replace=False)
    num_images = len(file_names)
    num_rows = int(np.ceil(num_images / images_per_row))

    # Create a figure with subplots
    fig, axes = plt.subplots(num_rows, images_per_row, figsize=(images_per_row * 4, num_rows * 4), squeeze=False)
    if num_rows == 1:
        axes = np.array(axes).reshape(1, -1)

    for i, file_name in enumerate(file_names):
        # Get the image ID from df_images using file_name
        img_data = df_images[df_images['file_name'] == file_name].iloc[0]
        image_id = img_data['id']

        # Load the image
        full_image_path = f'{image_path}/{file_name}'
        img = Image.open(full_image_path)

        # Filter the annotations for this image
        img_ann = df_annotations[df_annotations['image_id'] == image_id]

        # Select the correct subplot
        ax = axes[i // images_per_row, i % images_per_row]

        # Show the image
        ax.imshow(img)

        # Plot each bounding box
        for _, row in img_ann.iterrows():
            # COCO stores bounding boxes as [x, y, width, height]
            bbox = row['bbox']
            x, y, width, height = bbox
            category_id = row['category_id']

            # Set rectangle color based on category (you can customize it as needed)
            edge_color = 'r' if category_id == 1 else 'g'

            # Create and add the rectangle to the plot
            rect = patches.Rectangle((x, y), width, height, linewidth=1, edgecolor=edge_color, facecolor='none')
            ax.add_patch(rect)

            # Optionally add a label (category or any other info)
            ax.text(x, y - 10, f'Category {category_id}', color=edge_color, fontsize=10, weight='bold')

        # Remove axis for better visualization
        ax.axis('off')
        # Set the title with the file name
        ax.set_title(file_name, fontsize=8)

    # Turn off any unused subplots
    for j in range(num_images, num_rows * images_per_row):
        axes.flatten()[j].axis('off')

    # Adjust layout to prevent overlap and reduce spacing
    plt.subplots_adjust(wspace=0.1, hspace=0.1)
    plt.show()

In [ ]:
# Load COCO JSON file
with open('/content/car-license-plate-detection-iran/train/_annotations.coco.json', 'r') as f:
    coco_data = json.load(f)

# Convert annotations and images sections to DataFrames
df_annotations = pd.DataFrame(coco_data['annotations'])
df_images = pd.DataFrame(coco_data['images'])

# Ensure bbox is a proper list format
df_annotations['bbox'] = df_annotations['bbox'].apply(lambda x: x if isinstance(x, list) else json.loads(x))
plot_samples_from_coco(df_annotations, df_images, num_samples=6, image_path='/content/car-license-plate-detection-iran/train')

In [ ]:
!rm -r /content/car-license-plate-detection-iran_mscoco

In [ ]:
import os
import shutil

# Paths for source dataset and target dataset
src_dir = '/content/car-license-plate-detection-iran'
dst_dir = 'car-license-plate-detection-iran_mscoco'

# Define paths for images and annotations
images_dir = os.path.join(dst_dir, 'images')
annotations_dir = os.path.join(dst_dir, 'annotations')

# Create target directories for images and annotations
os.makedirs(os.path.join(images_dir, 'train'), exist_ok=True)
os.makedirs(os.path.join(images_dir, 'test'), exist_ok=True)
os.makedirs(os.path.join(images_dir, 'valid'), exist_ok=True)
os.makedirs(annotations_dir, exist_ok=True)

# Mapping of original folders to new paths
splits = ['train', 'test', 'valid']

# Loop over each split and copy files
for split in splits:
    src_split_dir = os.path.join(src_dir, split)
    dst_split_dir = os.path.join(images_dir, split)

    # Copy all image files
    for item in os.listdir(src_split_dir):
        item_path = os.path.join(src_split_dir, item)
        if os.path.isfile(item_path) and item.endswith(('.jpg', '.jpeg', '.png')):
            shutil.copy(item_path, os.path.join(dst_split_dir, item))
        elif item == '_annotations.coco.json':
            # Copy and rename the annotations file
            shutil.copy(item_path, os.path.join(annotations_dir, f'{split}.json'))

print("Dataset conversion complete with copied files!")


In [ ]:
import os
import zipfile

def zip_dataset(folder_path, output_zip):
    # Create the zip file
    with zipfile.ZipFile(output_zip, 'w', zipfile.ZIP_DEFLATED) as zipf:
        # Walk through the directory
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                # Create the correct file path within the zip file
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, os.path.dirname(folder_path))
                zipf.write(file_path, arcname=arcname)

# Paths
folder_path = '/content/car-license-plate-detection-iran_mscoco'  # Replace with the actual path to the folder
output_zip = 'License_detection.zip'

# Create zip
zip_dataset(folder_path, output_zip)
print(f"{output_zip} created successfully.")

# Import the Dataset to the project


1.   Upload the dataset
2.   Start a data import action
3.   Check the status of the dataset import action
4.   Check the processed dataset information

Upload the dataset

For uploading the dataset you have 2 options:-


*   Upload the dataset to cloud storage bucket with `project.upload_cloud_dataset()`
*   Upload the dataset from your local storage with `project.upload_local_dataset() `

In [ ]:
dataset, dataset_action = project_instance.import_local_dataset(
    dataset_name='Licence Plate detection',
    file_path = r"/content/License_detection.zip",
    dataset_type = "detection",
    dataset_description = "Licence plate detection dataset",
    version_description = "1st version",
    input_type = "image"
)

In [ ]:
dataset_id = dataset.dataset_id
action_id = dataset_action.action_id

print(f"Dataset ID: {dataset_id}")
print(f"Action ID: {action_id}")

In [ ]:
Vars.set("dataset_id", dataset.dataset_id)
Vars.set("dataset_action_id", dataset_action.action_id)

In [ ]:
import time
import pprint

action_id = dataset_action.action_id
def check_dataset_status():
    while True:
        print(" ")
        print("-------Status------")
        D = Dataset(session_instance, dataset_id=dataset_id)
        status = D.version_status
        print("Status of dataset:", status)
        print("-------------------")
        if status == 'processed':
            print("---------Preprocesing Complete---------")
            print("Dataset processed, proceed with experiment creation.")
            print("---------------------------------------")
            break

        time.sleep(90)  # Wait for 90 seconds before checking again

# Run the function to check dataset status
check_dataset_status()


In [ ]:
def print_dataset_info(dataset):

    dataset_id = dataset.dataset_details['_id']
    dataset_name = dataset.dataset_details['name']
    version_status = dataset.dataset_details.get("stats", [{}])[0].get("versionStatus")
    latest_version = dataset.dataset_details['latestVersion']
    no_of_samples = sum(version['versionStats']['total'] for version in dataset.dataset_details.get('stats', []))
    no_of_classes = len(dataset.dataset_details.get('stats', [{}])[0].get('classStat', {}))
    no_of_versions = len(dataset.dataset_details.get('allVersions', []))
    last_updated_at = dataset.dataset_details.get('updatedAt')

    print(f"Dataset ID: {dataset_id}")
    print(f"Dataset Name: {dataset_name}")
    print(f"Version Status: {version_status}")
    print(f"Latest Version: {latest_version}")
    print(f"Number of Samples: {no_of_samples}")
    print(f"Number of Classes: {no_of_classes}")
    print(f"Number of Versions: {no_of_versions}")
    print(f"Last Updated At: {last_updated_at}")

dataset.refresh()
print_dataset_info(dataset)

In [ ]:
import pprint
pprint.pprint(dataset.summary)
dataset_summary = dataset.summary

In [ ]:
import plotly.graph_objects as go
import pandas as pd
import plotly.io as pio


# Convert the histogram data to a DataFrame
df = pd.DataFrame(dataset_summary['histogram'])

# Bar chart for counts by category
fig1 = go.Figure(data=[go.Bar(x=df['label'], y=df['count'])])
fig1.update_layout(
    title='Count of Items by Category',
    xaxis_title='Category',
    yaxis_title='Count'
)

# Pie chart for distribution of data items
fig2 = go.Figure(data=[go.Pie(
    labels=['Test', 'Train', 'Unassigned', 'Validation'],
    values=[dataset_summary['testDataItemCount'], dataset_summary['trainDataItemCount'], dataset_summary['unassignedDataItemCount'], dataset_summary['valDataItemCount']],
    hole=0.3
)])
fig2.update_layout(
    title='Distribution of Data Items'
)

# Display the figures
fig1.show()
fig2.show()


# Create a Model Training experiment

1.   Start a Model Training experiment
2.   Explore the available SOTA model families and models
3.   Get the default training config , modify it if needed.
4.   Add the model for training.
5.   Check for the status of the model train action

In [ ]:
import os
import json
import pprint
import uuid

# Generate a unique experiment name
experiment_name_user = f"Vehicle-detection-experiment-{uuid.uuid4()}"

# Function to create an experiment
def create_experiment():
    name = experiment_name_user
    target_run_time = ["PyTorch"]
    primary_metric = "precision"
    matrice_compute = True
    dataset_version = 'v1.0'

    experiment_instance = project_instance.create_experiment(
        name,
        dataset_id,
        target_run_time[0],
        dataset_version,
        primary_metric,
        matrice_compute
    )

    print(experiment_instance)
    experiment_id = experiment_instance.experiment_id
    experiment_name = experiment_instance.experiment_name

    print(f"Experiment ID: {experiment_id}")
    print(f"Experiment Name: {experiment_name}")

    return experiment_instance, experiment_id, experiment_name

# Create a new experiment
experiment_instance , experiment_id, experiment_name = create_experiment()

print("----------------------------")
print(f"Experiment ID: {experiment_id}")
print(f"Experiment Name: {experiment_name}")
print(f"Experiment instance has been created: {experiment_instance}")
print("----------------------------")

In [ ]:
Vars.set("experiment_id", experiment_instance.experiment_id)
Vars.set("experiment_name", experiment_instance.experiment_name)

In [ ]:
from matrice.model_store import _get_all_model_families

resp , error , mesage = _get_all_model_families( session_instance , project_id , project_type="detection" )
print("----List of available classification models on platform----")
# Iterate through each model family in the response
for model_family in resp:
    # Extract _id and modelFamily (model family name)
    model_family_id = model_family.get('_id')
    name = model_family.get('modelFamily')

    # Print in formatted manner
    print(f"ID: {model_family_id}, Model Family Name: {name}")

In [ ]:
from matrice.model_store import ModelFamily

model_family_id = "65c677355f34a0e8878e3f05"
# Initialize the ModelFamily instance after choosing a Model Family
model_family_instance = ModelFamily(session_instance, model_family_id=model_family_id) # Update with the model family ID of your choice
print("A ModelFamily instance has been initialized",model_family_instance)

In [ ]:
# Fetch model architectures
arch_resp, error, message = model_family_instance.get_model_archs()
if error:
    print(f"Error: {error}")
else:

    # Check if resp is a list or a dictionary
    if isinstance(arch_resp, list):
        # Iterate through each model info in the list
        for model_info in arch_resp:
            model_key = model_info["model_key"]
            model_arch_instance = model_info["model_arch_instance"]

            # Extract model information
            model_info_id = model_arch_instance.model_info_id
            name = model_arch_instance.model_name
            key = model_arch_instance.model_key
            params = model_arch_instance.params_millions

            # Print in formatted manner
            print(f"ID: {model_info_id} |  Model Name: {name} | Model Key: {key} | Params in Millions: {params}")
    elif isinstance(arch_resp, dict):
        # Iterate through each model key in the dictionary
        for model_key, model_arch_instance in arch_resp.items():
            # Extract model information
            model_info_id = model_arch_instance.model_info_id
            name = model_arch_instance.model_name
            key = model_arch_instance.model_key
            params = model_arch_instance.params_millions

            # Print in formatted manner
            print(f"ID: {model_info_id} | Model Name: {name} | Model Key: {key} | Params in Millions: {params}")

In [ ]:
from matrice.model_store import ModelArch
train_arch = 'yolov8m' # Update with the model key of your choice
print("Chosen Training Architecture :" ,train_arch)
model_arch_instance = arch_resp.get(train_arch)
print("Model Architecture instance initialized for chosen training architecture :", model_arch_instance)

In [ ]:
import pprint
train_instance , train_config = model_arch_instance.get_train_config(experiment_id=Vars.get('experiment_id'))
print("-----Default Train Config------")
pprint.pprint(train_config)
print("--------------------------------")
print("-------Training instance initialized--------")
print(train_instance)

In [ ]:
model_config = train_config['model_config']

# Modify the model_config as needed
# modifications:
model_config['batch_size'] = [4]
model_config['epochs'] = [100]
model_config['learning_rate'] = [0.001]


# Repass the modified model_config into the train_config
train_config['model_config'] = model_config

# Print the updated train_config
print("-----Updated Model Config------")
pprint.pprint(train_config['model_config'])
print("--------------------------------")

# Print the updated train_config
print("-----Updated Train Config------")
pprint.pprint(train_config)
print("--------------------------------")

In [ ]:
train_resp , msg , err = experiment_instance.add_models_for_training(train_instance,train_config)
print("------------Model added for training----------")
pprint.pprint(train_resp)

In [ ]:
import time
from matrice.action import Action

# Define the action ID and the status you want to check
action_id = train_resp[0]['_idActionStatus']
success_status = 'SUCCESS'

# Function to check the status of the action
def check_action_status(S, action_id):
    action_instance = Action(S, action_id)
    print("---------------------")
    print(f"Action ID: {action_id}")
    print(f"Current Status: {action_instance.status}")
    print(f"Step Code: {action_instance.step_code}")
    print(f"Action service is : {action_instance.service_name}")
    print("---------------------")
    return action_instance.status

# Loop to check status every 2 minutes until it is 'success'
while True:
    status = check_action_status(session_instance, action_id)
    if status == success_status:
        print("Action status is 'success'. Model is successfully trained.")
        break
    else:
        print("Model is training. Waiting 10 minutes before checking again.")
        time.sleep(200)  # Wait for 10 minutes (600 seconds)


# Check the performance of the Trained Model

*   Check the performance values
*   Plot the epochs details of the model
*   Get the evaluation results of the model
*   Downlaod the trained model

In [ ]:
model_id = train_resp[0]['_id']
print(f"Model ID from response data: {model_id}")


# Initialize the Model class with the model_id
model_instance = Model(session_instance, model_id)
print("A Model instance has been initialized : ", model_instance)

In [ ]:
print(model_instance)

In [ ]:
Vars.set("model_train_id", model_id)

In [ ]:
# Print the test score, validation score, and best epoch
print(f"Test Score: {model_instance.test_score}")
print(f"Validation Score: {model_instance.val_score}")
print(f"Best Epoch: {model_instance.best_epoch}")

In [ ]:
model_instance.plot_epochs_losses()

In [ ]:
model_instance.plot_epochs_metrics()

In [ ]:
model_instance.plot_eval_results()

## Get test predictions on random samples from test set from the trained model

## Detection Pred

In [ ]:
import os
import random
import requests
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches


# Define the folder path containing the images
folder_path = r"/content/images_test"


# Get a list of all image files in the folder
image_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(('.jpg', '.jpeg', '.png'))]

# Randomly select 10 images from the folder
random_images = random.sample(image_files, 1)


# Loop through the selected images and send the prediction request
for image_file in random_images:

    result, error, message = model_instance.get_prediction(image_file)

    if error:
        print(f"Error: {error}")
        continue

    # Extract prediction data from the result
    predictions = result

    # Open the image
    image = Image.open(image_file)
    plt.figure(figsize=(10, 10))
    fig, ax = plt.subplots(1)
    ax.imshow(image)

    # Draw bounding boxes and labels
    for prediction in predictions:
        category = prediction["category"]
        confidence = prediction["confidence"]
        bbox = prediction["bounding_box"]

        # Extract the bounding box coordinates
        xmin, ymin, xmax, ymax = bbox["xmin"], bbox["ymin"], bbox["xmax"], bbox["ymax"]
        width, height = xmax - xmin, ymax - ymin

        # Create a Rectangle patch
        rect = plt.Rectangle((xmin, ymin), width, height, linewidth=2, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
        plt.text(xmin, ymin, f"{category} ({confidence:.2f})", bbox=dict(facecolor='yellow', alpha=0.5))

    plt.show()

In [ ]:
model_instance.download_model(file_name="model.pt")

# Export the model based on usage and platform and optimzie the Model Export


1.   Create a model export action using the export config
2.   Check the status of the model export action
3.   Download the exported model

In [ ]:
export_config = train_instance.get_default_export_config("ONNX") # Get the default export config for export format user wants (here ONNX)
print("Export Config for ONNX:")
pprint.pprint(export_config)

In [ ]:
exported_instance , action_instance = project_instance.create_model_export(model_id,"ONNX",export_config)

print("Model Export class has been initialized :", exported_instance)
print("----------------------------")
print(f"Export ID: {exported_instance.model_export_id}")
print(f"Export Name: {exported_instance.model_export_name}")
print(f"Action ID: {action_instance.action_id}")
print(f"Action Status: {action_instance.status}")
print("----------------------------")

In [ ]:
Vars.set("model_export_id", exported_instance.model_export_id)
Vars.set("model_export_name", exported_instance.model_export_name)

In [ ]:
import time
from matrice.action import Action

# Define the action ID and the status you want to check
action_id = action_instance.action_id
success_status = 'SUCCESS'

# Function to check the status of the action
def check_action_status(S, action_id):
    A = Action(S, action_id)
    print("---------------------")
    print(f"Action ID: {action_id}")
    print(f"Current Status: {A.status}")
    print(f"Step Code: {A.step_code}")
    print(f"Action service is : {A.service_name}")
    print("---------------------")
    return A.status

# Loop to check status every 1.5 minutes until it is 'success'
while True:
    status = check_action_status(session_instance, action_id)
    if status == success_status:
        print("Action status is 'success'. Model is successfully exported.")
        break
    else:
        print("Model is exporting. Waiting 2 minutes before checking again.")
        time.sleep(90)  # Wait for 1.5 minutes (90 seconds)


In [ ]:
exported_instance.download_model(file_name="model.onnx")

# Deploy the Model on Cloud

1.   Start a Model Deployment server
2.   Make Predictions on the Deployed Model

In [ ]:
deployment , deployment_action = project_instance.create_deployment(
    model_id = Vars.get('model_train_id'),  # It can also be model_train_id depending on the model you want to deploy
    deployment_name = "vehicle-detection",
    shutdown_threshold=20
)

In [ ]:
key_resp = deployment.create_auth_key(10)
auth_key = key_resp[0]

In [ ]:
Vars.set("deployment_id", deployment.deployment_id)
Vars.set("deployment_name", deployment.deployment_name)
Vars.set("auth_key", auth_key)

In [ ]:
import os
import random
import requests
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches


# Define the folder path containing the images
folder_path = r"/content/images_test"


# Get a list of all image files in the folder
image_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(('.jpg', '.jpeg', '.png'))]

# Randomly select 10 images from the folder
random_images = random.sample(image_files, 1)


# Loop through the selected images and send the prediction request
for image_file in random_images:
    # Check if image_file is a valid path
    if not isinstance(image_file, str):
        print(f"Invalid image path: {image_file}")
        continue

    # Send prediction request
    result, error, message = deployment.get_prediction(auth_key, image_file)

    if error:
        print(f"Error: {error}")
        continue

    # Extract prediction data from the result
    predictions = result

    # Open the image
    image = Image.open(image_file)
    plt.figure(figsize=(10, 10))
    fig, ax = plt.subplots(1)
    ax.imshow(image)

    # Draw bounding boxes and labels
    for prediction in predictions:
        category = prediction["category"]
        confidence = prediction["confidence"]
        bbox = prediction["bounding_box"]

        # Extract the bounding box coordinates
        xmin, ymin, xmax, ymax = bbox["xmin"], bbox["ymin"], bbox["xmax"], bbox["ymax"]
        width, height = xmax - xmin, ymax - ymin

        # Create a Rectangle patch
        rect = plt.Rectangle((xmin, ymin), width, height, linewidth=2, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
        plt.text(xmin, ymin, f"{category} ({confidence:.2f})", bbox=dict(facecolor='yellow', alpha=0.5))

    plt.show()